# Documentation
Author: Randi Ang

The goal is to align vector readings (X-Y-Z axes) readings from an accelerometer (values in m/s^2) to a common plane of reference (in this case the Earth) using orientation information from the gyroscope (values in degrees). The gimbal point would be at the point of origin (0,0,0). This method uses 3D matrix rotation to find the rotated coordinates using the original XYZ values and rotation values (Pitch-Roll-Yaw).

Phone orientation:
1. Azimuth (Yaw): The direction (north/south/east/west) the device is pointing. 0 is magnetic north.

2. Pitch: The top-to-bottom tilt of the device. 0 is flat, levelled to the ground, screen facing up. 

3. Roll: The left-to-right tilt of the device. 0 is flat, levelled to the ground, screen facing up.

4. X axis: short side (width of phone) --> +ve value (move right); -ve values (move left)

5. Y axis: long side (length of phone) --> +ve value (move front); -ve values (move back)

6. Z axis: front and back side of the phone --> +ve value (move up); -ve values (move down)

A change in the +ve or -ve values represents both a change in direction and magnitude of acceleration

It is recommended to take the linear acceleration values by default, which would remove the effect of gravity. If the device is orientated to a neutral position on a perfectly levelled surface, gravity would apply a relatively constant acceleration of +9.8m/s^2 on the z-axis.

This formula does not make adjustments for gravity. However, this may be adjusted for by taking the "Rotated-Z minus 9.8" (Z3 - 9.8).
Earth's gravity at the Poles = 9.832 m/s^2
Earth's gravity at the Equator = 9.789 m/s^2

At this stage we're not sure what is the sensitivity involved, so we'll use a constant of 9.8 m/s^2. But if you want a better accuracy for SG (which is near the equator), a constant of 9.789 m/s^2 should give a more accurate reading.

For iOS devices, the acceleration on the X-Y-Z axes are given in Gs. 
1 g = 9.80665 m/s^2

## References
https://keisan.casio.com/exec/system/15362817755710
https://www.universetoday.com/26775/gravity-of-the-earth/

In [1]:
# Import Library
import numpy as np
import pandas as pd

In [2]:
# Initialise Set-Up
df = pd.DataFrame(
    [
        (10,  1, 5, 90, 0, 0),
        (10,  1, 5, 0, 90, 0),
        (10,  1, 5, 0, 0, 90),
        (10,  1, 5, 90, 90, 90)
    ],
    columns=['X', 'Y', 'Z', 'Pitch', 'Roll', 'Azimuth']
)
df.head()

,X,Y,Z,Pitch,Roll,Azimuth
0,10,1,5,90,0,0
1,10,1,5,0,90,0
2,10,1,5,0,0,90
3,10,1,5,90,90,90


In [3]:
# Inspect data type
# In reality the xyz coordinate values and Pitch-Roll-Azimuth values would be floats not integers
df.dtypes

X          int64
Y          int64
Z          int64
Pitch      int64
Roll       int64
Azimuth    int64
dtype: object

In [4]:
# Step 1
# Define function

def rotationCorrection(X0 = 0, Y0 = 0, Z0 = 0, Pitch = 0, Roll = 0, Azimuth = 0):
    # The goal is to realign the X-Y-Z axes to a gimbal point at (0,0,0)
    # with the device perfectly levelled  and oriented correctly to the ground
    # this function is designed to work with dataframes
    
    # Key packages used:
    # Pandas as pd
    # Numpy as np
    
    # Input Variables
    # X0 = orginal x coordinate value (if values are from Android accelerometer, values are in m/s^2)
    # Y0 = orginal Y coordinate value (if values are from Android accelerometer, values are in m/s^2)
    # Z0 = orginal Z coordinate value (if values are from Android accelerometer, values are in m/s^2)
    # Pitch = Rotation about x-axis in degrees from gyroscope
    # Roll = Rotation about y-axis in degrees from gyroscope
    # Azimuth (aka Yaw) = Rotation about z-axis in degrees from gyroscope
    
    # Notes:
    # Coordinate values from X-Y-Z axes must be on the same scale
    # Rotation values (pitch, roll, azimuth) must be on the same scale
    # All values are required, though a value of 0 (0 acceleration and/or rotation) by default
    
    # Reverse Angles in Degrees
    # subsequent functions require values to be in radians format so a conversion is required
    alpha = np.radians(-Pitch) 
    beta = np.radians(-Roll)
    delta = np.radians(-Azimuth)
    
    # Rotation Sequence: X --> Y --> Z
    
    # Rotation about X-Axis (Pitch)
    X1 = X0 # Optional; for clarity sake, possible to remove to reduce memory requirements
    Y1 = np.cos(alpha) * Y0 - np.sin(alpha) * Z0
    Z1 = np.sin(alpha) * Y0 + np.cos(alpha) *Z0
    
    # Rotation about Y-Axis (Roll)
    X2 = np.cos(beta) * X1 + np.sin(beta) * Z1 # Alt form (less memory): np.cos(beta) * X0 + np.sin(beta) * Z1
    Y2 = Y1 # Optional; for clarity sake, possible to remove to reduce memory requirements
    Z2 = -np.sin(beta) * X1 + np.cos(beta) * Z1 # Alt form (less memory): -np.sin(beta) * X0 + np.cos(beta) * Z1
    
    # Rotation about Z-Axis (Azimuth)
    X3 = np.cos(delta) * X2 - np.sin(delta) * Y2 # Alt form (less memory): np.cos(delta) * X2 - np.sin(delta) * Y1
    Y3 = np.sin(delta) * X2 + np.cos(delta) * Y2 # Alt form (less memory): np.sin(delta) * X2 + cos(delta) * Y1
    Z3 = Z2 # Optional; for clarity sake, possible to remove to reduce memory requirements
    
    # Return rotated values as an array
    rotatedval_complete = np.array((X3,Y3,Z3))
    
    return rotatedval_complete

In [5]:
# Step 2
# Calculate the rotated values

# DO NOT CHANGE THE INDEX OF THE DATAFRAME WHEN PERFORMING THIS STEP OR THE CELLS WOULD BE MAPPED WRONGLY!!!!

# Get output of rotated values as an array
rotatedVal_array = rotationCorrection(X0 = df['X'], Y0 = df['Y'], Z0 = df['Z'], Pitch = df['Pitch'], Roll = df['Roll'], Azimuth = df['Azimuth'])

# Test results (Diagnostic Test only)
rotatedVal_array[0:3,0:2]

array([[10., -5.],
       [ 5.,  1.],
       [-1., 10.]])

In [6]:
# Step 3
# Add rotated values in array format to data frame

# DO NOT CHANGE THE INDEX OF THE DATAFRAME WHEN PERFORMING THIS STEP OR THE CELLS WOULD BE MAPPED WRONGLY!!!!

df["Rotated-X"] = rotatedVal_array[0]
df["Rotated-Y"] = rotatedVal_array[1]
df["Rotated-Z"] = rotatedVal_array[2]

# Delete rotatedVal_array to save memory as we won't be using it anymore
del rotatedVal_array

# Inspect Dataframe to check if rotated values has been added (Diagnostic Test only)
df.head()

,X,Y,Z,Pitch,Roll,Azimuth,Rotated-X,Rotated-Y,Rotated-Z
0,10,1,5,90,0,0,10.0,5.0,-1.0
1,10,1,5,0,90,0,-5.0,1.0,10.0
2,10,1,5,0,0,90,1.0,-10.0,5.0
3,10,1,5,90,90,90,5.0,-1.0,10.0


In [7]:
# Inspect data type of newly added values in dataframe
# In reality the xyz coordinate values and Pitch-Roll-Azimuth values would be floats not integers
df.dtypes

X              int64
Y              int64
Z              int64
Pitch          int64
Roll           int64
Azimuth        int64
Rotated-X    float64
Rotated-Y    float64
Rotated-Z    float64
dtype: object

# End